In [3]:
import pandas as pd
import numpy as np

In [ ]:
df =pd.read_csv("C:\\Users\\ACCESS\\Documents\\event-management\\Superstore Sales.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df = df.drop("Postal Code", axis=1)

In [ ]:
df["Order Date"] = pd.to_datetime(df["Order Date"], errors="coerce", dayfirst=True)
df["Ship Date"]  = pd.to_datetime(df["Ship Date"], errors="coerce", dayfirst=True)


In [ ]:
df[['Order Date', 'Ship Date']].head()

In [ ]:
wrong_dates=df[df['Ship Date'] <= df['Order Date']]

In [7]:
df["shipping_duration_days"] = (df["Ship Date"] - df["Order Date"]).dt.days

In [8]:
df[["Order Date", "Ship Date", "shipping_duration_days"]].head(10)

,Order Date,Ship Date,shipping_duration_days
0,2017-11-08,2017-11-11,3
1,2017-11-08,2017-11-11,3
2,2017-06-12,2017-06-16,4
3,2016-10-11,2016-10-18,7
4,2016-10-11,2016-10-18,7
5,2015-06-09,2015-06-14,5
6,2015-06-09,2015-06-14,5
7,2015-06-09,2015-06-14,5
8,2015-06-09,2015-06-14,5
9,2015-06-09,2015-06-14,5


In [9]:
negative = df[df["shipping_duration_days"] < 0]
negative.head()


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Region,Product ID,Category,Sub-Category,Product Name,Sales,shipping_duration_days


In [ ]:

conflicted_ids = (
    df.groupby("Product ID")["Product Name"]
    .nunique()
)
conflicted_ids = conflicted_ids[conflicted_ids > 1].index


In [ ]:

conflict_details = (
    df[df["Product ID"].isin(conflicted_ids)]
    [["Product ID", "Product Name"]]
    .drop_duplicates()
    .sort_values("Product ID")
)

conflict_details


,Product ID,Product Name
2471,FUR-BO-10002213,"Sauder Forest Hills Library, Woodland Oak Finish"
2115,FUR-BO-10002213,DMI Eclipse Executive Suite Bookcases
66,FUR-CH-10001146,"Global Value Mid-Back Manager's Chair, Gray"
128,FUR-CH-10001146,"Global Task Chair, Black"
1459,FUR-FU-10001473,DAX Wood Document Frame
...,...,...
1219,TEC-PH-10002200,Samsung Galaxy Note 2
2596,TEC-PH-10002310,Plantronics Calisto P620-M USB Wireless Speake...
1378,TEC-PH-10002310,Panasonic KX T7731-B Digital phone
922,TEC-PH-10004531,OtterBox Commuter Series Case - iPhone 5 & 5s


In [ ]:
name_map = (
    df.groupby("Product ID")["Product Name"]
    .apply(lambda x: x.mode()[0])  
    .to_dict()
)

In [17]:

df["Product Name"] = df["Product ID"].map(name_map)

In [18]:
df.groupby("Product ID")["Product Name"].nunique().sort_values(ascending=False).head(10)


Product ID
TEC-PH-10004977    1
FUR-BO-10000112    1
FUR-BO-10000330    1
FUR-BO-10000362    1
FUR-BO-10000468    1
TEC-PH-10004539    1
TEC-PH-10004536    1
TEC-PH-10004531    1
TEC-PH-10004522    1
TEC-PH-10004447    1
Name: Product Name, dtype: int64

In [19]:
df.groupby("Product ID")["Product Name"].nunique().reset_index().query("`Product Name` > 1")


,Product ID,Product Name


In [20]:
long_ship = df[df["shipping_duration_days"] > 60]
long_ship.head()


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Region,Product ID,Category,Sub-Category,Product Name,Sales,shipping_duration_days


In [21]:
df[df['Sales'] < 0]

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Region,Product ID,Category,Sub-Category,Product Name,Sales,shipping_duration_days


In [22]:
df[df['shipping_duration_days'] < 0]
df['shipping_duration_days'].describe()

count    9800.000000
mean        3.961122
std         1.749614
min         0.000000
25%         3.000000
50%         4.000000
75%         5.000000
max         7.000000
Name: shipping_duration_days, dtype: float64

In [23]:
df['shipping_duration_days'].astype(int)

0       3
1       3
2       4
3       7
4       7
       ..
9795    7
9796    5
9797    5
9798    5
9799    5
Name: shipping_duration_days, Length: 9800, dtype: int64

In [24]:
df.duplicated().sum()

np.int64(0)

In [25]:
df['Sales'].describe()


count     9800.000000
mean       230.769059
std        626.651875
min          0.444000
25%         17.248000
50%         54.490000
75%        210.605000
max      22638.480000
Name: Sales, dtype: float64

In [27]:
df['Sales'] = df['Sales'].round(2)

In [28]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_')


In [9]:
 mul= df.groupby("product_name")["product_id"] .nunique() 
 mul=mul[mul>1]

In [10]:
mul

product_name
#10- 4 1/8" x 9 1/2" Recycled Envelopes           2
Avery Non-Stick Binders                           2
Easy-staple paper                                 8
Eldon Wave Desk Accessories                       2
KI Adjustable-Height Table                        2
Okidata C610n Printer                             2
Peel & Seel Recycled Catalog Envelopes, Brown     2
Prang Drawing Pencil Set                          2
Staple envelope                                   9
Staple holder                                     3
Staple magnet                                     2
Staple remover                                    3
Staple-based wall hangings                        2
Staples                                          10
Staples in misc. colors                           7
Storex Dura Pro Binders                           2
Name: product_id, dtype: int64

In [14]:
id_map = df.groupby("product_name")["product_id"].agg(lambda x: x.mode()[0])
df["product_id"] = df["product_name"].map(id_map)

In [15]:
no_mul= df.groupby("product_name")["product_id"] .nunique() 
no_mul=no_mul[no_mul>1]

In [16]:
no_mul

Series([], Name: product_id, dtype: int64)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   row_id                  9800 non-null   int64         
 1   order_id                9800 non-null   object        
 2   order_date              9800 non-null   datetime64[ns]
 3   ship_date               9800 non-null   datetime64[ns]
 4   ship_mode               9800 non-null   object        
 5   customer_id             9800 non-null   object        
 6   customer_name           9800 non-null   object        
 7   segment                 9800 non-null   object        
 8   country                 9800 non-null   object        
 9   city                    9800 non-null   object        
 10  state                   9800 non-null   object        
 11  region                  9800 non-null   object        
 12  product_id              9800 non-null   object  

In [17]:
df.to_csv("C:\\Users\\ACCESS\\Documents\\event-management\\Superstore Sales.csv", index=False)